# Costos y Beneficios asociados a la implementación del SMI por Comuna (Análisis instantáneo)

Primero se realiza el cálculo de los costos de implementación de SMI en Chile y luego se estiman los beneficios. Para esto se toma como base la información proveniente de la tabla *df_caracterizacion_distribucion*, la cual se ha obtenido a partir de su procesamiento en "*Caracterizacion_Dx_Dic2024_comuna (CNE).ipynb*".

In [1]:
import pandas as pd 
import numpy as np
import unidecode as ud
from pprint import pprint

In [2]:
df_caracterizacion_distribucion = pd.read_excel("./Para obtener la caracterización de Dx/Salidas/Caracterizacion_Dx_Dic2024_comuna (CNE).xlsx")

# Dolar y UF considerados (a 16 de Oct de 2025, según SII)
usd = 959.36
uf 	= 39521.26

# Se setea la comuna y el segmento como índice del DataFrame
df_caracterizacion_distribucion = df_caracterizacion_distribucion.set_index(["Comuna", "Segmento"], drop=False)

# Se anualiza la Energía facturada
df_caracterizacion_distribucion["Energía promedio mensual [kWh]"] = (df_caracterizacion_distribucion["Energía promedio mensual [kWh]"] * 12) / 1000  # Conversión a MWh/año
df_caracterizacion_distribucion["Energía facturada [USD]"] = df_caracterizacion_distribucion["Energía facturada [USD]"] * 12

# Se eliminan columnas que no serán utilizadas
df_caracterizacion_distribucion = df_caracterizacion_distribucion.drop(columns=["Cantidad de alimentadores", "Energía facturada [USD]"])

# Se renombran columnas para mayor claridad
df_caracterizacion_distribucion = df_caracterizacion_distribucion.rename(columns={
	"Energía promedio mensual [kWh]": "Energía facturada anual [MWh/año]",
	"Energía facturada [USD]": "Energía facturada anual [USD/año]"
})

crecimiento_anual_demanda = 0.0302  			# Crecimiento anual de la demanda (3.02%)

crecimiento_anual_residencial = 0.0153  		# Crecimiento anual de clientes residenciales (BT) (1.53%)
crecimiento_anual_no_residencial = 0.0094  		# Crecimiento anual de clientes no residenciales (AT) (0.94%)
crecimiento_anual_libredx = 0.0315  			# Crecimiento anual de clientes libre Dx (3.15%)

# Los datos utilizados de energía y cantidad de clientes corresponden al año 2024 (df_implementacion), sin embargo,
# el año base de la implementación es 2025 (df_implementacion_0). Salvo para el caso de lo clientes libres el cual si está actualizado a 2025.
factor_crecimiento_cantidad_clientes = np.where(df_caracterizacion_distribucion["Segmento"] == 'Residencial', crecimiento_anual_residencial,
											np.where(df_caracterizacion_distribucion["Segmento"] == 'No Residencial BT', crecimiento_anual_residencial,
												np.where(df_caracterizacion_distribucion["Segmento"] == 'No Residencial AT', crecimiento_anual_no_residencial, 0)))

factor_crecimiento_demanda = np.where(df_caracterizacion_distribucion["Segmento"] == 'Residencial', crecimiento_anual_demanda,
								np.where(df_caracterizacion_distribucion["Segmento"] == 'No Residencial BT', crecimiento_anual_demanda,
									np.where(df_caracterizacion_distribucion["Segmento"] == 'No Residencial AT', crecimiento_anual_demanda, 0)))

# Se actualizan los valores de "Cantidad de clientes" según el crecimiento anual esperado (factor compuesto)
df_caracterizacion_distribucion["Cantidad de clientes"] = ((df_caracterizacion_distribucion["Cantidad de clientes"]
															* (1 + factor_crecimiento_cantidad_clientes)).round().astype('Int64'))

# Se actualiza la Energía facturada anual según el crecimiento anual esperado (factor compuesto)
df_caracterizacion_distribucion["Energía facturada anual [MWh/año]"] = (df_caracterizacion_distribucion["Energía facturada anual [MWh/año]"]
																		* (1 + factor_crecimiento_demanda))

df_caracterizacion_distribucion

Comuna           Segmento  Densidad  \
Comuna      Segmento                                                      
chimbarongo Residencial        chimbarongo        Residencial  MUY BAJA   
            No Residencial BT  chimbarongo  No Residencial BT  MUY BAJA   
            No Residencial AT  chimbarongo  No Residencial AT  MUY BAJA   
            LibreDx            chimbarongo            LibreDx  MUY BAJA   
            Red                chimbarongo                Red  MUY BAJA   
...                                    ...                ...       ...   
valdivia    Residencial           valdivia        Residencial      BAJA   
            No Residencial BT     valdivia  No Residencial BT      BAJA   
            No Residencial AT     valdivia  No Residencial AT      BAJA   
            LibreDx               valdivia            LibreDx      BAJA   
            Red                   valdivia                Red      BAJA   

                              Tipo de facturación  Superficie efectiva [km2]  \
Comuna      Segmento                                                           
chimbarongo Residencial                   Mensual                       0.00   
            No Residencial BT             Mensual                       0.00   
            No Residencial AT             Mensual                       0.00   
            LibreDx                       Mensual                       0.00   
            Red                           Mensual                     130.00   
...                                           ...                        ...   
valdivia    Residencial                 Bimensual                       0.00   
            No Residencial BT           Bimensual                       0.00   
            No Residencial AT           Bimensual                       0.00   
            LibreDx                     Bimensual                       0.00   
            Red                         Bimensual                     242.75   

                               Cantidad de clientes  UM 1F para TD  \
Comuna      Segmento                                                 
chimbarongo Residencial                       14053              0   
            No Residencial BT                   413              0   
            No Residencial AT                   298              0   
            LibreDx                               0              0   
            Red                                   0             33   
...                                             ...            ...   
valdivia    Residencial                       73312              0   
            No Residencial BT                  1565              0   
            No Residencial AT                   568              0   
            LibreDx                               0              0   
            Red                                   0            342   

                               UM 3F 1S para TD  UM 3F 2S para TD  \
Comuna      Segmento                                                
chimbarongo Residencial                       0                 0   
            No Residencial BT                 0                 0   
            No Residencial AT                 0                 0   
            LibreDx                           0                 0   
            Red                              66               221   
...                                         ...               ...   
valdivia    Residencial                       0                 0   
            No Residencial BT                 0                 0   
            No Residencial AT                 0                 0   
            LibreDx                           0                 0   
            Red                             274              1043   

                               Energía facturada anual [MWh/año]  \
Comuna      Segmento                                               
chimbarongo Residencial                             29186.868709   
      

In [3]:
# Información al mes de diciembre de 2024
# -> Cantidad de clientes.
# -> Cantidad de energía.
cantidad_clientes = df_caracterizacion_distribucion['Cantidad de clientes'].sum()
cantidad_energia = df_caracterizacion_distribucion['Energía facturada anual [MWh/año]'].sum()

df_facturacion_clientes_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'Residencial']
cantidad_clientes_residenciales = df_facturacion_clientes_residenciales['Cantidad de clientes'].sum()
cantidad_energia_residenciales = df_facturacion_clientes_residenciales['Energía facturada anual [MWh/año]'].sum()

df_facturacion_clientes_no_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'No Residencial BT']
cantidad_clientes_no_residenciales_bt = df_facturacion_clientes_no_residenciales['Cantidad de clientes'].sum()
cantidad_energia_no_residenciales_bt = df_facturacion_clientes_no_residenciales['Energía facturada anual [MWh/año]'].sum()

df_facturacion_clientes_no_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'No Residencial AT']
cantidad_clientes_no_residenciales_at = df_facturacion_clientes_no_residenciales['Cantidad de clientes'].sum()
cantidad_energia_no_residenciales_at = df_facturacion_clientes_no_residenciales['Energía facturada anual [MWh/año]'].sum()

df_facturacion_clientes_no_residenciales = df_caracterizacion_distribucion[df_caracterizacion_distribucion['Segmento'] == 'LibreDx']
cantidad_clientes_no_residenciales_libredx = df_facturacion_clientes_no_residenciales['Cantidad de clientes'].sum()
cantidad_energia_no_residenciales_libredx = df_facturacion_clientes_no_residenciales['Energía facturada anual [MWh/año]'].sum()

cantidad_um_1f_td = df_caracterizacion_distribucion['UM 1F para TD'].sum()
cantidad_um_3f1s_td = df_caracterizacion_distribucion['UM 3F 1S para TD'].sum()
cantidad_um_3f2s_td = df_caracterizacion_distribucion['UM 3F 2S para TD'].sum()

print("(A fecha de diciembre de 2024)",
      "\nClientes regulados:", 
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes), 
      "\n -> Energía facturada: {:,} GWh".format((cantidad_energia / 1_000).round(2)),
      "\nClientes Residenciales:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_residenciales),
      "\n -> Energía facturada: {:,} GWh".format((cantidad_energia_residenciales / 1_000).round(2)),
      "\nClientes No Residenciales en BT:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales_bt),
      "\n -> Energía facturada: {:,} GWh".format((cantidad_energia_no_residenciales_bt / 1_000).round(2)),
      "\nClientes No Residenciales en AT:",
	  "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales_at),
	  "\n -> Energía facturada: {:,} GWh".format((cantidad_energia_no_residenciales_at / 1_000).round(2)),
      "\nClientes No Residenciales LibreDx:",
      "\n -> Cantidad de clientes: {:,}".format(cantidad_clientes_no_residenciales_libredx),
      "\n -> Energía facturada: {:,} GWh".format((cantidad_energia_no_residenciales_libredx / 1_000).round(2)),
      "\n--------------------------------------------",
      "\nCantidad de UM para TD:",
	  "\n -> UM 1F para TD: {:,}".format(cantidad_um_1f_td),
	  "\n -> UM 3F 1S para TD: {:,}".format(cantidad_um_3f1s_td),
	  "\n -> UM 3F 2S para TD: {:,}".format(cantidad_um_3f2s_td),
	  "\n")

(A fecha de diciembre de 2024) 
Clientes regulados: 
 -> Cantidad de clientes: 7,745,291 
 -> Energía facturada: 34,765.77 GWh 
Clientes Residenciales: 
 -> Cantidad de clientes: 7,543,340 
 -> Energía facturada: 16,993.97 GWh 
Clientes No Residenciales en BT: 
 -> Cantidad de clientes: 145,001 
 -> Energía facturada: 4,513.76 GWh 
Clientes No Residenciales en AT: 
 -> Cantidad de clientes: 56,524 
 -> Energía facturada: 6,161.01 GWh 
Clientes No Residenciales LibreDx: 
 -> Cantidad de clientes: 426 
 -> Energía facturada: 7,097.03 GWh 
-------------------------------------------- 
Cantidad de UM para TD: 
 -> UM 1F para TD: 22,259 
 -> UM 3F 1S para TD: 28,854 
 -> UM 3F 2S para TD: 93,120 



## Definición del tipo de tecnología a utilizar en función de la densidad

In [12]:
# Decisión de la tecnología a ocupar; PLC, Celular, RF, LoRa, etc.
df_requerimientos = df_caracterizacion_distribucion.copy()

# Se definen los niveles de densidad que existen en la tabla
niveles_densidad = [
    "EXTREMADAMENTE BAJA",
    "MUY BAJA",
    "BAJA",
    "MEDIA",
    "ALTA"
]

print("Se asigna la tecnología de comunicaciones para cada nivel de densidad")
print("Tecnologías posibles: 'Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa'")
print("----------------------------------------------------")

# Input de tecnología y armando del mapeo
mapeo_tecnologia = {}

for densidad in niveles_densidad:
    tecnologia = input(f"Tecnología para una densidad '{densidad}': ")
    if tecnologia not in ['Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa']:
        while True:
            print("Inválida. Tecnologías posibles: 'Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa'.")
            tecnologia = input(f"Tecnología para una densidad '{densidad}': ")
            if tecnologia in ['Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa']:
                break
    mapeo_tecnologia[densidad] = tecnologia
    
print("Tecnologías asignadas por nivel de densidad:")
for densidad, tecnologia in mapeo_tecnologia.items():
    print(f" - {densidad}: {tecnologia}")

# Se crea la nueva columna usando la columna 'Densidad'
df_requerimientos["Tecnología"] = df_requerimientos["Densidad"].map(mapeo_tecnologia)

Se asigna la tecnología de comunicaciones para cada nivel de densidad
Tecnologías posibles: 'Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa'
----------------------------------------------------
Tecnologías asignadas por nivel de densidad:
 - EXTREMADAMENTE BAJA: RF-Mesh
 - MUY BAJA: RF-Mesh
 - BAJA: RF-Mesh
 - MEDIA: RF-Mesh
 - ALTA: RF-Mesh


## Cálculo sobre la cantidad de equipos necesarios por comuna y segmento

Supuestos considerados para asignar las UM necesarias en cada segmento:

 - Cliente Residencial + TD 1F ....................... -> Se le asigna la Unidad de Medida (UM) 1F.
 - Cliente No Residencial ........................... -> Se le asigna la Unidad de Medida (UM) 3F de conexión directa (sin CT).
 - TD 3F 1S + TD 3F 2S + Cliente libre ............-> Se le asigna la Unidad de Medida (UM) 3F de conexión indirecta (con CT).

### Requerimientos de Unidades de Medida (UM)

In [13]:
# Se agrega la columna de "Cantidad UM 1F" según el segmento
valores = []

for i, row in df_requerimientos.iterrows():
    
    if row['Segmento'] == 'Residencial' or row['Segmento'] == 'No Residencial BT':
        valores.append(row['Cantidad de clientes'])
    elif row['Segmento'] == 'Red':
        valores.append(row['UM 1F para TD'])
    else:
        valores.append(0)

df_requerimientos["Cantidad UM 1F"] = pd.Series(valores, index=df_requerimientos.index).astype('Int64')

# Se agrega la columna de "Cantidad UM 3F Directa" según el segmento
valores = []

for i, row in df_requerimientos.iterrows():
    
    if row['Segmento'] == 'No Residencial AT':			# Supuesto
        valores.append(row['Cantidad de clientes'])
    else:
        valores.append(0)

df_requerimientos["Cantidad UM 3F Directa"] = pd.Series(valores, index=df_requerimientos.index).astype('Int64')

# Se agrega la columna de "Cantidad UM 3F Indirecta" según el segmento
valores = []

for i, row in df_requerimientos.iterrows():
    
    if row['Segmento'] == 'Red' or row['Segmento'] == 'LibreDx':
        valores.append(row['UM 3F 1S para TD'] + row['UM 3F 2S para TD'] * 2 + row['Cantidad de clientes'])
    else:
        valores.append(0)

df_requerimientos["Cantidad UM 3F Indirecta"] = pd.Series(valores, index=df_requerimientos.index).astype('Int64')
df_requerimientos

Comuna           Segmento  Densidad  \
Comuna      Segmento                                                      
chimbarongo Residencial        chimbarongo        Residencial  MUY BAJA   
            No Residencial BT  chimbarongo  No Residencial BT  MUY BAJA   
            No Residencial AT  chimbarongo  No Residencial AT  MUY BAJA   
            LibreDx            chimbarongo            LibreDx  MUY BAJA   
            Red                chimbarongo                Red  MUY BAJA   
...                                    ...                ...       ...   
valdivia    Residencial           valdivia        Residencial      BAJA   
            No Residencial BT     valdivia  No Residencial BT      BAJA   
            No Residencial AT     valdivia  No Residencial AT      BAJA   
            LibreDx               valdivia            LibreDx      BAJA   
            Red                   valdivia                Red      BAJA   

                              Tipo de facturación  Superficie efectiva [km2]  \
Comuna      Segmento                                                           
chimbarongo Residencial                   Mensual                       0.00   
            No Residencial BT             Mensual                       0.00   
            No Residencial AT             Mensual                       0.00   
            LibreDx                       Mensual                       0.00   
            Red                           Mensual                     130.00   
...                                           ...                        ...   
valdivia    Residencial                 Bimensual                       0.00   
            No Residencial BT           Bimensual                       0.00   
            No Residencial AT           Bimensual                       0.00   
            LibreDx                     Bimensual                       0.00   
            Red                         Bimensual                     242.75   

                               Cantidad de clientes  UM 1F para TD  \
Comuna      Segmento                                                 
chimbarongo Residencial                       14053              0   
            No Residencial BT                   413              0   
            No Residencial AT                   298              0   
            LibreDx                               0              0   
            Red                                   0             33   
...                                             ...            ...   
valdivia    Residencial                       73312              0   
            No Residencial BT                  1565              0   
            No Residencial AT                   568              0   
            LibreDx                               0              0   
            Red                                   0            342   

                               UM 3F 1S para TD  UM 3F 2S para TD  \
Comuna      Segmento                                                
chimbarongo Residencial                       0                 0   
            No Residencial BT                 0                 0   
            No Residencial AT                 0                 0   
            LibreDx                           0                 0   
            Red                              66               221   
...                                         ...               ...   
valdivia    Residencial                       0                 0   
            No Residencial BT                 0                 0   
            No Residencial AT                 0                 0   
            LibreDx                           0                 0   
            Red                             274              1043   

                               Energía facturada anual [MWh/año]  \
Comuna      Segmento                                               
chimbarongo Residencial                             29198.201214   
      

### Requerimientos de Unidades Concentradoras (DCU)

Supuestos considerados para las DCU de RF y LoRa:
 - Tecnología RF-Mesh: Cobertura de 800 metros a la redonda (extendible a través de UM) y soporta 1000 dispositivos.
 - Tecnología LoRa: Cobertura de **3.5km** a la redonda en zona urbana (zonas Alta y Media) y **17.5km** en rural (ie, zonas Baja, Muy Baja y Extremadamente Baja).

In [ ]:
# Se calclula la cantidad de Unidades Concentradoras (DCU) necesarias
valores = []

for i, row in df_requerimientos.iterrows():
    
    if row["Tecnología"] == "G3-PLC" and row["Segmento"] == "Red":
        dcu_requeridas = (row['UM 1F para TD'] + row['UM 3F 1S para TD'] + row['UM 3F 2S para TD'])		# Sujeto a cambios según estimación de TD
        valores.append(dcu_requeridas)
    
    elif row["Tecnología"] == "RF-Mesh" and row["Segmento"] == "Red":
        capacidad = 1000  																								# Capacidad de 1000 equipos por DCU
        cantidad_cliente_comuna = df_requerimientos.loc[(row["Comuna"], slice(None)), "Cantidad de clientes"].sum()			
        dcu_requeridas = (cantidad_cliente_comuna // capacidad) + 1  													# Redondeo hacia arriba
        valores.append(dcu_requeridas)

    elif row["Tecnología"] == "LoRa" and row["Segmento"] == "Red":
        if row["Densidad"] == "EXTREMADAMENTE BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "BAJA":
            cobertura_km2 = (15 ** 2) * np.pi 
            dcu_requeridas = np.ceil(row["Superficie efectiva [km2]"] / cobertura_km2)					# Rural
            valores.append(dcu_requeridas) 	
        else:
            cobertura_km2 = (2 ** 2) * np.pi
            dcu_requeridas = np.ceil(row["Superficie efectiva [km2]"] / cobertura_km2)					# Urbano
            valores.append(dcu_requeridas)
            
    elif row["Tecnología"] == "TWACS" and row["Segmento"] == "Red":										# Caso TWACS: 1 DCU por SPD
        dcu_requeridas = row["Cantidad SPD"]							
        valores.append(dcu_requeridas)

    else:        
        dcu_necesarias = 0								# Caso de Celular y otros segmentos
        valores.append(dcu_necesarias)

df_requerimientos["Cantidad DCU"] = pd.Series(valores, index=df_requerimientos.index).astype('Int64')
df_requerimientos

Comuna           Segmento  Densidad  \
Comuna      Segmento                                                      
chimbarongo Residencial        chimbarongo        Residencial  MUY BAJA   
            No Residencial BT  chimbarongo  No Residencial BT  MUY BAJA   
            No Residencial AT  chimbarongo  No Residencial AT  MUY BAJA   
            LibreDx            chimbarongo            LibreDx  MUY BAJA   
            Red                chimbarongo                Red  MUY BAJA   
...                                    ...                ...       ...   
valdivia    Residencial           valdivia        Residencial      BAJA   
            No Residencial BT     valdivia  No Residencial BT      BAJA   
            No Residencial AT     valdivia  No Residencial AT      BAJA   
            LibreDx               valdivia            LibreDx      BAJA   
            Red                   valdivia                Red      BAJA   

                              Tipo de facturación  Superficie efectiva [km2]  \
Comuna      Segmento                                                           
chimbarongo Residencial                   Mensual                       0.00   
            No Residencial BT             Mensual                       0.00   
            No Residencial AT             Mensual                       0.00   
            LibreDx                       Mensual                       0.00   
            Red                           Mensual                     130.00   
...                                           ...                        ...   
valdivia    Residencial                 Bimensual                       0.00   
            No Residencial BT           Bimensual                       0.00   
            No Residencial AT           Bimensual                       0.00   
            LibreDx                     Bimensual                       0.00   
            Red                         Bimensual                     242.75   

                               Cantidad de clientes  UM 1F para TD  \
Comuna      Segmento                                                 
chimbarongo Residencial                       14053              0   
            No Residencial BT                   413              0   
            No Residencial AT                   298              0   
            LibreDx                               0              0   
            Red                                   0             33   
...                                             ...            ...   
valdivia    Residencial                       73312              0   
            No Residencial BT                  1565              0   
            No Residencial AT                   568              0   
            LibreDx                               0              0   
            Red                                   0            342   

                               UM 3F 1S para TD  UM 3F 2S para TD  \
Comuna      Segmento                                                
chimbarongo Residencial                       0                 0   
            No Residencial BT                 0                 0   
            No Residencial AT                 0                 0   
            LibreDx                           0                 0   
            Red                              66               221   
...                                         ...               ...   
valdivia    Residencial                       0                 0   
            No Residencial BT                 0                 0   
            No Residencial AT                 0                 0   
            LibreDx                           0                 0   
            Red                             274              1043   

                               Energía facturada anual [MWh/año]  \
Comuna      Segmento                                               
chimbarongo Residencial                             29198.201214   
      

### Requerimientos de comunicaciones para el tramo DCU -> SGO

Se calcula la cantidad de datos requeridos a transferir entre el DCU y el SGO, considerando mediciones, alarmas, actualizaciones y control (Estudio del [Anexo 4-1, VAD 2024-2028](https://comisionenergia-my.sharepoint.com/:x:/r/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/Doc.aspx?sourcedoc=%7B8873AC9E-7FED-4D32-95FF-28A4F39C3069%7D&file=Anexo%204-10%20-CNE-SM-COSTOS%20ITP_final.xlsx&action=default&mobileredirect=true)). Estas cantidades dependen del tipo de tecnología a utilizar. Además, se han utilizado precios provenientes de una cotización realizada con Entel para el mismo proceso VAD.

Se ha considerado:
- 128,2 kB es el requerimiento **mensual** de datos para 1 cliente.

In [16]:
# Se calcula la cantidad de datos transmitida por DCU (Cálulos realizados en 'Costos.xlsx')

# 1: Se consideran las exigencias de medición de la Norma SMMC hasta antes del 2031 (Artículo 9-8)
# 0: Se consideran las exigencias de medición de la Norma SMMC desde el 2031 en adelante (Artículo 9-8)
exigencias_de_medicion = 1		# Input: 1 o 0

if exigencias_de_medicion == 1:
	datos_cliente_3f_zona_alta = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_3f_zona_baja = 45.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_alta = 71.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_baja = 32.5 / 1024		# MB mensual por cliente

	datos_trafo_3f_zona_alta = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_3f_zona_baja = 45.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_baja = 38.5 / 1024		# MB mensual por trafo

else:
	datos_cliente_3f_zona_alta = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_3f_zona_baja = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_alta = 71.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_baja = 71.7 / 1024		# MB mensual por cliente

	datos_trafo_3f_zona_alta = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_3f_zona_baja = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_baja = 83.7 / 1024		# MB mensual por trafo


valores = []

for i, row in df_requerimientos.iloc[4::5].iterrows():			# Recorrer solo las filas de segmento "Red"		
	cantidad_dcu = df_requerimientos.loc[i, "Cantidad DCU"]

	# Se suman las cantidades correspondientes de UM enfocadas a clientes y transformadores según la densidad de la zona
	if cantidad_dcu > 0 and (row["Densidad"] == "ALTA" or row["Densidad"] == "MEDIA"):
		total_clientes_3f_zona_alta = df_requerimientos.loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
		total_clientes_1f_zona_alta = df_requerimientos.loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

		total_trafo_3f_zona_alta = (df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
									+ df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
		total_trafo_1f_zona_alta = df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

		mb_exigidos = (datos_cliente_3f_zona_alta * total_clientes_3f_zona_alta + datos_cliente_1f_zona_alta * total_clientes_1f_zona_alta
								+ datos_trafo_3f_zona_alta * total_trafo_3f_zona_alta + datos_trafo_1f_zona_alta * total_trafo_1f_zona_alta) / cantidad_dcu
		valores.append(mb_exigidos)

	elif cantidad_dcu > 0 and (row["Densidad"] == "BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "EXTREMADAMENTE BAJA"):
		total_clientes_3f_zona_baja = df_requerimientos.loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
		total_clientes_1f_zona_baja = df_requerimientos.loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

		total_trafo_3f_zona_baja = (df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
									+ df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
		total_trafo_1f_zona_baja = df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

		mb_exigidos = (datos_cliente_3f_zona_baja * total_clientes_3f_zona_baja + datos_cliente_1f_zona_baja * total_clientes_1f_zona_baja
								+ datos_trafo_3f_zona_baja * total_trafo_3f_zona_baja + datos_trafo_1f_zona_baja * total_trafo_1f_zona_baja) / cantidad_dcu
		valores.append(mb_exigidos)

	else:        
		mb_exigidos = 0								# Caso de Celular, TWACS y otros segmentos
		valores.append(mb_exigidos)
		
df_requerimientos["MB Mensuales por DCU"] = 0.0
filtro = df_requerimientos["Segmento"] == "Red"

df_requerimientos.loc[filtro, "MB Mensuales por DCU"] = pd.Series(valores, index=df_requerimientos.index[filtro]).round(4)

In [ ]:
# df_productividad = pd.read_excel("Proyecciones_y_Estimaciones.xlsx", sheet_name="Productividad Instalación")
# df_productividad = df_productividad.set_index(["Densidad", "Equipo"], drop=False)			# Se asignan indices

# cantidad_de_cuadrillas = input("Cantidad de cuadrillas (por comuna) para la implementación: ")

# df_productividad["Productividad [Equipos/día]"] = (df_productividad["Productividad [Equipos/día]"] * 5 * 4 * 12 * int(cantidad_de_cuadrillas))
# df_productividad.rename(columns={"Productividad [Equipos/día]": "Productividad [Equipos/año]"}, inplace=True)
# df_productividad

# # Se calcula la productividad anual global que se puede alcanzar
# valores = []

# for i, row in df_requerimientos.iterrows():
#     segmento = row["Segmento"]
#     densidad = row["Densidad"]
    
#     if segmento == "Residencial":
#         productividad = df_productividad.loc[(densidad, "UM 1F"), "Productividad [Equipos/año]"]
#         if productividad > row["Cantidad UM 1F"]:
#               productividad = row["Cantidad UM 1F"]
#         valores.append(productividad)
        
#     elif segmento == "No_Residencial":
#         productividad = df_productividad.loc[(densidad, "UM 3F Directa"), "Productividad [Equipos/año]"]
#         if productividad > row["Cantidad UM 3F Directa"]:
#             productividad = row["Cantidad UM 3F Directa"]
#         valores.append(productividad)
        
#     elif segmento == "LibreDx":
#         productividad = df_productividad.loc[(densidad, "UM 3F Indirecta"), "Productividad [Equipos/año]"]
#         if productividad > row["Cantidad UM 3F Indirecta"]:
#             productividad = row["Cantidad UM 3F Indirecta"]
#         valores.append(productividad)
        
#     else:
#         productividad_3f = df_productividad.loc[(densidad, "UM 3F Indirecta"), "Productividad [Equipos/año]"]
#         productividad_dcu = df_productividad.loc[(densidad, "DCU"), "Productividad [Equipos/año]"]
#         if productividad_3f > row["Cantidad UM 1F"] + row["Cantidad UM 3F Indirecta"]:
#             productividad_3f = row["Cantidad UM 1F"] + row["Cantidad UM 3F Indirecta"]
#         if productividad_dcu > row["Cantidad DCU"]:
#             productividad_dcu = row["Cantidad DCU"]
        
#         productividad = productividad_3f + productividad_dcu
#         valores.append(productividad)
        
# df_requerimientos["Productividad Anual [Equipos/año]"] = pd.Series(valores, index=df_requerimientos.index).astype('Int64')

# df_requerimientos["Productividad Anual [Equipos/año]"].sum()

## Cálculo sobre los costos de inversión por comuna y segmento

Para este primer escenario se han considerado precios de lista de los equipos e instalación, sin descuentos por volumen. Posteriormente se tomará en cuenta esto.

In [16]:
df_capex_smi = pd.read_excel("Costos.xlsx", sheet_name="CAPEX", skiprows=4, usecols="B:M")
df_capex_smi = df_capex_smi.set_index(["Tecnología", "Densidad"], drop=False)			# Se asignan indices

# Cálculo de la inversión
df_inversion = df_requerimientos.copy()

descuento_por_volumen = float(input("Descuento por volumen [%]: ")) / 100      # Definicion del descuento por volumen

inversion_um_1f = []
inversion_um_3f_dir = []
inversion_um_3f_ind = []
inversion_dcu = []
inversion_sgo = []
inversion_mdms = []

for i, row in df_inversion.iterrows():
	costo_um_1f = (df_capex_smi.loc[(row["Tecnología"], row["Densidad"]), "UM 1F [USD/u]"] * (1 - descuento_por_volumen))
	costo_um_3f_dir = (df_capex_smi.loc[(row["Tecnología"], row["Densidad"]), "UM 3F Directa [USD/u]"] * (1 - descuento_por_volumen))
	costo_um_3f_ind = (df_capex_smi.loc[(row["Tecnología"], row["Densidad"]), "UM 3F Indirecta [USD/u]"] * (1 - descuento_por_volumen))
	costo_dcu = (df_capex_smi.loc[(row["Tecnología"], row["Densidad"]), "DCU [USD/u]"] * (1 - descuento_por_volumen))
	costo_sgo = df_capex_smi.loc[(row["Tecnología"], row["Densidad"]), "SGO [USD/Medidor]"]
	costo_mdms = df_capex_smi.loc[(row["Tecnología"], row["Densidad"]), "MDMS [USD/Medidor]"]

	instalacion_um_1f = (df_capex_smi.loc[(row["Tecnología"], row["Densidad"]), "Instalación UM 1F [USD/u]"] * (1 - descuento_por_volumen))
	instalacion_um_3f_dir = (df_capex_smi.loc[(row["Tecnología"], row["Densidad"]), "Instalación UM 3F Directa [USD/u]"] * (1 - descuento_por_volumen))
	instalacion_um_3f_ind = (df_capex_smi.loc[(row["Tecnología"], row["Densidad"]), "Instalación UM 3F Indirecta [USD/u]"] * (1 - descuento_por_volumen))
	instalacion_dcu = (df_capex_smi.loc[(row["Tecnología"], row["Densidad"]), "Instalación DCU [USD/u]"] * (1 - descuento_por_volumen))

	inv_um_1f = (row["Cantidad UM 1F"] * costo_um_1f + row["Cantidad UM 1F"] * instalacion_um_1f).round(2)
	inv_um_3f_dir = (row["Cantidad UM 3F Directa"] * costo_um_3f_dir + row["Cantidad UM 3F Directa"] * instalacion_um_3f_dir).round(2)
	inv_um_3f_ind = (row["Cantidad UM 3F Indirecta"] * costo_um_3f_ind + row["Cantidad UM 3F Indirecta"] * instalacion_um_3f_ind).round(2)
	inv_dcu = (row["Cantidad DCU"] * costo_dcu + row["Cantidad DCU"] * instalacion_dcu).round(2)
	inv_sgo = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * costo_sgo).round(2)
	inv_mdms = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * costo_mdms).round(2)

	inversion_um_1f.append(inv_um_1f)
	inversion_um_3f_dir.append(inv_um_3f_dir)
	inversion_um_3f_ind.append(inv_um_3f_ind)
	inversion_dcu.append(inv_dcu)
	inversion_sgo.append(inv_sgo)
	inversion_mdms.append(inv_mdms)

# Se agregan las columnas de inversión por equipo al DataFrame
df_inversion["Inversión UM 1F [USD]"] = pd.Series(inversion_um_1f, index=df_inversion.index)
df_inversion["Inversión UM 3F Directa [USD]"] = pd.Series(inversion_um_3f_dir, index=df_inversion.index)
df_inversion["Inversión UM 3F Indirecta [USD]"] = pd.Series(inversion_um_3f_ind, index=df_inversion.index)
df_inversion["Inversión DCU [USD]"] = pd.Series(inversion_dcu, index=df_inversion.index)
df_inversion["Inversión SGO [USD]"] = pd.Series(inversion_sgo, index=df_inversion.index)
df_inversion["Inversión MDMS [USD]"] = pd.Series(inversion_mdms, index=df_inversion.index)

# Se suma la inversión total por comuna y segmento
df_inversion["Inversión Total [USD]"] = (df_inversion["Inversión UM 1F [USD]"] + df_inversion["Inversión UM 3F Directa [USD]"] + 
										df_inversion["Inversión UM 3F Indirecta [USD]"] + df_inversion["Inversión DCU [USD]"] + 
                                        df_inversion["Inversión SGO [USD]"] + df_inversion["Inversión MDMS [USD]"])
df_inversion["Inversión Total [USD]"] = df_inversion["Inversión Total [USD]"].round(2)

# Inversión total 
print("Inversión total del proyecto dada la siguiente configuración de tecnologías:")
for densidad, tecnologia in mapeo_tecnologia.items():
    print(f" - {densidad}: {tecnologia}")
print("\nConsidera además un descuento por volumen de compra e instalación de UM: {:,.0f}%".format(descuento_por_volumen * 100))
print("----------------------------------------------------------------------------")
print("Inversión total: M USD {:,.2f}".format(df_inversion["Inversión Total [USD]"].sum() / 1_000_000))

Inversión total del proyecto dada la siguiente configuración de tecnologías:
 - EXTREMADAMENTE BAJA: RF-Mesh
 - MUY BAJA: RF-Mesh
 - BAJA: RF-Mesh
 - MEDIA: RF-Mesh
 - ALTA: RF-Mesh

Considera además un descuento por volumen de compra e instalación de UM: 20%
----------------------------------------------------------------------------
Inversión total: M USD 1,474.45


### Resultado de la inversión inicial de SMI en chile

In [17]:
# Para conocer la inversión de cada item por tecnología
suma_por_columna = df_inversion.iloc[:, 20:27].sum()

print("Inversión por segmento (en millones de dólares):", 
      "\n ->","\n -> ".join(f"{col[0:-6]} [MUSD]: {val / 1_000_000}" for col, val in suma_por_columna.items()))

Inversión por segmento (en millones de dólares): 
 -> Inversión UM 1F [MUSD]: 1169.4669665899999
 -> Inversión UM 3F Directa [MUSD]: 20.038248009999997
 -> Inversión UM 3F Indirecta [MUSD]: 126.12641606999999
 -> Inversión DCU [MUSD]: 0.60991638
 -> Inversión SGO [MUSD]: 145.44123575
 -> Inversión MDMS [MUSD]: 12.76426
 -> Inversión Total [MUSD]: 1474.4470428000002


### Inversión Total por Tipo de Densidad

In [18]:
# Se calcula la inversión necesaria separada por tipo de densidad

for densidad in niveles_densidad:
    df_densidad = df_inversion[df_inversion["Densidad"] == densidad]
    inversion_densidad = df_densidad["Inversión DCU [USD]"].sum()
    print(f"Inversión para densidad '{densidad}': M USD {inversion_densidad / 1_000_000:,.2f}")

Inversión para densidad 'EXTREMADAMENTE BAJA': M USD 0.21
Inversión para densidad 'MUY BAJA': M USD 0.09
Inversión para densidad 'BAJA': M USD 0.20
Inversión para densidad 'MEDIA': M USD 0.08
Inversión para densidad 'ALTA': M USD 0.03


## Cálculo Sobre los costos de operación por comuna y segmento

In [ ]:
df_opex_smi = pd.read_excel("Costos.xlsx", sheet_name="OPEX", skiprows=4, usecols="B:G")
df_opex_smi = df_opex_smi.set_index(["Tecnología", "Densidad"], drop=False)			        # Se asignan indices

# Planes de datos, cotización de entel a Dic 2022 (ajustado por IPC, factor=1.12)
plan_10_mb = (1200 / usd) * 1.12  			# Costo mensual en pesos chilenos para 10 MB
plan_10_mb_extra = (100 / usd) * 1.12  		# Costo mensual en pesos chilenos por cada 1 MB extra sobre el plan base
plan_200_mb = (2800 / usd) * 1.12  			# Costo mensual en pesos chilenos para 200 MB
plan_300_mb = (3300 / usd) * 1.12  			# Costo mensual en pesos chilenos para 300 MB
plan_200_300_mb_extra = (50 / usd) * 1.12  	# Costo mensual en pesos chilenos por cada 1 MB extra sobre el plan base

# Cálculo de la inversión
df_operacion = df_inversion.copy()

comunicaciones = []
comunicaciones_dcu_sgo = []
sgo = []
mdms = []
mantenimiento_equipos = []

for i, row in df_inversion.iterrows():
	opex_com = df_opex_smi.loc[(row["Tecnología"], row["Densidad"]), "Comunicaciones [USD/medidor]"] 
	opex_sgo = df_opex_smi.loc[(row["Tecnología"], row["Densidad"]), "Operación SGO [USD/medidor]"]
	opex_mdms = df_opex_smi.loc[(row["Tecnología"], row["Densidad"]), "Operación MDMS [USD/medidor]"]
	opex_mantenimiento_equipos = df_opex_smi.loc[(row["Tecnología"], row["Densidad"]), "Soporte y Mantenimiento Equipos [USD/medidor]"]

	operacion_com = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_com).round(2)
	operacion_sgo = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_sgo).round(2)
	operacion_mdms = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_mdms).round(2)
	operacion_mantenimiento_equipos = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_mantenimiento_equipos).round(2)
      
	mb_mensuales = row["MB Mensuales por DCU"]

	if mb_mensuales < 26:                   # Conveniencia económica de plan 10 MB
		operacion_com_dcu_sgo = (plan_10_mb + round(mb_mensuales - 10) * plan_10_mb_extra) * row["Cantidad DCU"] * 12
		comunicaciones_dcu_sgo.append(operacion_com_dcu_sgo)
	if 26 <= mb_mensuales <= 210:			# Conveniencia económica de plan 200 MB
		operacion_com_dcu_sgo = (plan_200_mb + round(mb_mensuales - 200) * plan_200_300_mb_extra) * row["Cantidad DCU"] * 12
		comunicaciones_dcu_sgo.append(operacion_com_dcu_sgo)
	if 210 < mb_mensuales:
		operacion_com_dcu_sgo = (plan_300_mb + round(mb_mensuales - 300) * plan_200_300_mb_extra) * row["Cantidad DCU"] * 12
		comunicaciones_dcu_sgo.append(operacion_com_dcu_sgo)

	comunicaciones.append(operacion_com)
	sgo.append(operacion_sgo)
	mdms.append(operacion_mdms)
	mantenimiento_equipos.append(operacion_mantenimiento_equipos)

# Se agregan las columnas de inversión por equipo al DataFrame
df_operacion["Operación Comunicaciones [USD/año]"] = pd.Series(comunicaciones, index=df_operacion.index)
df_operacion["Operacion Comunicaciones DCU-SGO [USD/año]"] = pd.Series(comunicaciones_dcu_sgo, index=df_operacion.index).astype("Float64")
df_operacion["Operación SGO [USD/año]"] = pd.Series(sgo, index=df_operacion.index)
df_operacion["Operación MDMS [USD/año]"] = pd.Series(mdms, index=df_operacion.index)
df_operacion["Soporte y Mantenimiento Equipos [USD/año]"] = pd.Series(mantenimiento_equipos, index=df_operacion.index)

# Se suma la inversión total por comuna y segmento
df_operacion["Costos Operacionales Totales [USD/año]"] = (df_operacion["Operación Comunicaciones [USD/año]"]
														   	+ df_operacion["Operacion Comunicaciones DCU-SGO [USD/año]"]
														   	+ df_operacion["Operación SGO [USD/año]"]
															+ df_operacion["Operación MDMS [USD/año]"]
															+ df_operacion["Soporte y Mantenimiento Equipos [USD/año]"]).round(2)

# Operacion total 
print("Costos de operación totales del proyecto dada la siguiente configuración de tecnologías:\n")
for densidad, tecnologia in mapeo_tecnologia.items():
    print(f" - {densidad}: {tecnologia}")
print("-----------------------------")
print("Costos de operación totales (anuales): M USD {:,.2f}".format(df_operacion["Costos Operacionales Totales [USD/año]"].sum() / 1_000_000))

### Resultado de los costos de operación y mantenimiento del SMI en chile

In [ ]:
# Para conocer la inversión de cada item por tecnología
suma_por_columna = df_operacion.iloc[:, 21:27].sum()

print("Inversión por segmento (en millones de dólares):", 
      "\n ->","\n -> ".join(f"{col[0:-10]} [MUSD/año]: {val / 1_000_000}" for col, val in suma_por_columna.items()))

In [ ]:
df_operacion = df_operacion.drop(columns=["Operación Comunicaciones [USD/año]", "Operacion Comunicaciones DCU-SGO [USD/año]", 
                                          "Operación SGO [USD/año]", "Operación MDMS [USD/año]", 
                                          "Soporte y Mantenimiento Equipos [USD/año]"])
df_operacion

## Cálculo sobre los beneficios esperados asociados a la implementación de SMI

In [ ]:
#### Algunas consideraciones y supuestos para el calculo de los beneficios:

# Precio de la energía (PNP equivalente nacional, Anexos PNP Julio 2025)
precio_energia_usd_mwh = 93.14   		# USD/MWh

# Compensaciones automáticas e instruidas por la SEC por fallas en el suministro eléctrico (Fuente: Informe SEC Dic 2024)
monto_compensado_anual = (8_744_000_000 + 3_961_000_000) * 1.037 / usd			# USD/año Automáticas + Instruidas (2024)
cantidad_de_compensaciones = 3_257_118 + 1_150_658								# N° Automáticas + Instruidas (2024)

# Costo de falla 2024 para el SEN bajo una profundidad de entre 0-5%
costo_de_falla_2024 = 415.60			# USD/MWh

# Cálculo de la energía no suministrada anual (fuente SEC)
energia_no_suministrada_anual = monto_compensado_anual / (2 * costo_de_falla_2024)	 	# MWh/año

# Unidades de CO2 emitidas por kWh generado (Fuente: Estudio Ministerio del Medio Ambiente)
emisiones_co2_por_mwh = 0.2421		# tCO2/MWh

# Costo social de emitir toneladas de CO2 (Fuente: Ministerio de Desarrollo Social)
costo_social_co2 = 63.4			# USD/tCO2

### Beneficio 1: Ahorro por efecto de la reducción en la forma de lectura pedestre de la energía

Este corresponde al beneficio más directo y perceptible, pues en virtud de la telemedición es posible ahorrar el 100% de los costos incurridos en medición pedestre de energía (informe GTD).

Para la estimación de los costos se hizo uso de la información proveniente de los anexos del VAD ([Anexo 4-i, VAD 2024-2028](https://comisionenergia-my.sharepoint.com/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fonedrive%2Dsubdeptargedis%5Fcne%5Fcl%2FDocuments%2FPublicaciones%20WEB%2FVAD%2FVAD%202024%2D2028%2F02%20Informe%20T%C3%A9cnico%20Preliminar%2FATD1%2FAnexos%2FAnexo%204%2FAnexo%204%2Di%20Atencion%20al%20Cliente&viewid=f5258698%2D4137%2D4785%2Da047%2D925c3d159fd9&ga=1)) sobre los costos de medición por punto de medición dependiente del ATD.

In [ ]:
# Se calcula el ahorro anual por reducción en la lectura pedestre
while True:
    lectura_remota = float(input("Reducción en la lectura pedestre [%]: ")) / 100
    if 0 <= lectura_remota <= 1:
	    break

# Costo de lectura pedestre por medición (Datos del proceso VAD 2024-2028, 'Beneficios.xlsx')
costos_por_densidad = {
    "ALTA": 0.20416,					# USD/medición
    "MEDIA": 0.2653,					# USD/medición
    "BAJA": 0.4133,					    # USD/medición
    "MUY BAJA": 0.7136,					# USD/medición
    "EXTREMADAMENTE BAJA": 0.8853		# USD/medición	
}

# Se considera entonces:
# Medición mensual: costo_lectura * 12
# Medición bimensual: costo_lectura * 6

# Costo base por registro (por medición)
costo_base = df_operacion["Densidad"].map(costos_por_densidad)

# Factor anual según periodicidad: mensual = 12, bimensual = 6
factor_anual = np.where(df_operacion["Tipo de facturación"] == "Mensual", 12,
               np.where(df_operacion["Tipo de facturación"] == "Bimensual", 6, np.nan))

# Indice para aplicar el cálculo solo al segmento Regulado (Residencial y No_Residencial)
indice_segmento = np.where(df_operacion["Segmento"] == "Residencial", 1,
               		np.where(df_operacion["Segmento"] == "No Residencial BT", 1,
                        np.where(df_operacion["Segmento"] == "No Residencial AT", 1, 0)))

# Nueva columna con costo anual
df_operacion["Ahorro en lectura pedestre [USD/año]"] = (costo_base
                                                        * factor_anual
														* df_operacion["Cantidad de clientes"]
														* indice_segmento
                                                        * lectura_remota)

beneficio_neto_telemedicion = df_operacion["Ahorro en lectura pedestre [USD/año]"].sum()

# Ahorro global anual por efectos de la telemedición
print(f"Considera la reducción de un {lectura_remota * 100}% de lectura pedestre")
print("-------------------------------------------------------------")
print("Ahorro por reducción de lectura pedestre (anual): M USD {:,.4f}"
	  .format(beneficio_neto_telemedicion / 1_000_000))

### Beneficio 2: Ahorro por efecto de la reducción del corte y reposición del suministro remoto

Este beneficio se calcula a través de costo anual destinado a corte y reposición establecido en el VAD ([Anexos VAD 2024-2028](https://comisionenergia-my.sharepoint.com/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fonedrive%2Dsubdeptargedis%5Fcne%5Fcl%2FDocuments%2FPublicaciones%20WEB%2FVAD%2FVAD%202024%2D2028%2F02%20Informe%20T%C3%A9cnico%20Preliminar%2FATD4%2FAnexos%2FAnexo%204%2FAnexo%204%2D1&viewid=f5258698%2D4137%2D4785%2Da047%2D925c3d159fd9&ga=1)) asociado al ATD4 (basado en la empresa Saesa). Este monto se divide entre la cantidad de clientes de Saesa, y se obtiene entonces el monto anual por cliente.

El informe de GTD afirma que se puede eliminar los costes de corte y reposición en un 81%.

In [ ]:
# Ahorros asociados al corte y reposición remotos de suministro
while True:
	cortes_remotos = float(input("Reducción de corte y reposición de suministro pedestre [%]: ")) / 100
	if 0 <= cortes_remotos <= 1:
	    break

# Costo promedio por corte e reposición de suministro (Fuente: VAD 2024-2028)
corte_y_repo = 0.4534		# USD/cliente-año

# Indice para aplicar el cálculo solo al segmento Regulado (Residencial y No_Residencial)
indice_segmento = np.where(df_operacion["Segmento"] == "Residencial", 1,
               		np.where(df_operacion["Segmento"] == "No Residencial BT", 1,
                        np.where(df_operacion["Segmento"] == "No Residencial AT", 1, 0)))

# Ahorro anual por reducción en cortes y reposición remotos de suministro
df_operacion["Ahorro corte y reposición remoto [USD/año]"] = (df_operacion["Cantidad de clientes"]
															  * corte_y_repo
															  * cortes_remotos
															  * indice_segmento)

beneficio_neto_corte_repo = df_operacion["Ahorro corte y reposición remoto [USD/año]"].sum()

# Ahorro global anual por efectos de la telemedición
print(f"Considera la reducción de un {cortes_remotos * 100}% en cortes y reposición de suministro")
print("-------------------------------------------------------------")
print("Ahorro por reducción en corte y reposición remotos (anual): M USD {:,.4f}"
	  .format(beneficio_neto_corte_repo / 1_000_000))

### Beneficio 3: Ahorro por efecto de la reducción de las pérdidas No técnicas

Tomando en cuenta que las pérdidas (tanto técnicas como No técnicas) están calculadas sobre la energía comprada, se debe seguir la siguiente ecuación para el cálculo de esta energía:

$\text{Energía Comprada [MWh]} = \frac{\text{Energía Vendida [MWh]}}{1 - PT - PH - PC}$   

donde, 
 - PT: Pérdidas técnicas 
 - PH: Pérdidas por hurto
 - PC: Pérdidas Comerciales

UK afirma que, en términos generales, se podría reducir el Hurto en un 10% siendo pesimistas, alcanzando reducciones de entre un 20-30% en ciertos escenarios ([Despliegue de UK](https://assets.publishing.service.gov.uk/media/5d7f54c4e5274a27c2c6d53a/smart-meter-roll-out-cost-benefit-analysis-2019.pdf)).

GTD afirma que esta reducción en el Hurto podría ser en torno a un 35% (Informe GTD 1).

Por otro lado, en cuanto a la reducción en las pérdidas comerciales, considerando que estas tienen que ver con la pérdida por falta de mantenimiento de medidores (como se detalla  acá [Resolución Exenta VAD 2024-2028](https://www.cne.cl/wp-content/uploads/2023/01/ResEx-CNE-N%C2%B0-29-Aprueba-BT-Corregidas-VAD-24-28_20-01-23.pdf), esta se podría llegar a reducir en torno a un 50% tomando en cuenta que la clase de precisión de los MI es mayor y que en su mayoría cuentan con autodiagnóstico.

Por otro lado, para estimar el ahorro que supondría, se tomará como precio de la energía el PNP equivalente nacional calculado en el proceso PNP Julio 2025 ([Anexos PNP Julio 2025](https://comisionenergia-my.sharepoint.com/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fonedrive%2Dsubdeptargedis%5Fcne%5Fcl%2FDocuments%2FPublicaciones%20WEB%2FPNP%2F2025%2D02%2F02%20ITD%2F4%2E%2DModelo&viewid=f5258698%2D4137%2D4785%2Da047%2D925c3d159fd9&ga=1)).

In [ ]:
# Se calcula el ahorro anual por reducción del hurto de energía
while True:
    porcentaje_reduccion_hurto = float(input("Reducción en el hurto de energía [%]: ")) / 100
    porcentaje_reduccion_comercial = float(input("Reducción en las pérdidas comerciales [%]: ")) / 100
    if 0 <= porcentaje_reduccion_hurto <= 1 and 0 <= porcentaje_reduccion_comercial <= 1:
	    break

# Perdidas técnicas (pt), pérdidas por hurto (ph) y pérdidas comerciales (pc) nominales
pt = df_operacion["Pérdidas de energía [pu]"]
ph = df_operacion["Hurto de energía [pu]"]
pc = df_operacion["Pérdidas comerciales [pu]"]

# Cálculo de las pérdidas reducidas
hurto_reducido = df_operacion["Hurto de energía [pu]"] * porcentaje_reduccion_hurto
comercial_reducido = df_operacion["Pérdidas comerciales [pu]"] * porcentaje_reduccion_comercial

# Se calcula de esa forma dado que los % están sobre la energía comprada, no la facturada
df_operacion["Ahorro reducción de PNT [USD/año]"] = ((((df_operacion["Energía facturada anual [MWh/año]"] / (1 - pt - ph - pc))
                                                        * hurto_reducido)
                                                        * precio_energia_usd_mwh) +
													   (((df_operacion["Energía facturada anual [MWh/año]"] / (1 - pt - ph - pc))
                                                        * comercial_reducido)
                                                        * precio_energia_usd_mwh))

beneficio_neto_reduccion_pnt = df_operacion["Ahorro reducción de PNT [USD/año]"].sum()

print(f"Considera la reducción de un {porcentaje_reduccion_hurto * 100}% en el hurto de energía")
print(f"Considera la reducción de un {porcentaje_reduccion_comercial * 100}% en las pérdidas comerciales")
print("---------------------------------------------------------------")
print("Ahorro por reducción de PNT (anual): M USD {:,.4f}"
	  .format(beneficio_neto_reduccion_pnt / 1_000_000))

### Beneficio 4: Ahorro por efecto de la reducción del tiempo de aviso y localización de fallas

Para el cálculo de este beneficio se ha usado el monto compensado neto en 2024 que indica la SEC ([Informe SEC Dic 2024](https://www.sec.cl/sitio-web/wp-content/uploads/2025/02/Informe-SEC-Diciembre-2024.pdf)). A partir de este monto se busca la energía no suministrada en base a la siguiente formula definida por la SEC [Compensaciones Dx](https://www.sec.cl/sitio-web/wp-content/uploads/2019/06/ANEXO_PROCESO_COMPENSACIONES_DX-1.pdf):

$\text{Monto a Compensar [USD/año]} = 2 * \text{Energía No Suministrada [MWh/año]} * \text{Costo de Falla [USD/MWh]}$

Para el costo de falla se considera el costo de falla para el SEN bajo una profundidad de entre 0-5% (según [Costos de falla 2024](https://www.diariooficial.interior.gob.cl/publicaciones/2024/09/03/43941/01/2540114.pdf) y [Costos de falla 2025](https://www.cne.cl/wp-content/uploads/2025/10/Rex-503-comunica-CFCD-y-CFLD.pdf))

En cuanto a la reducción promedio del tiempo de No Suministro, [Despliegue UK](https://assets.publishing.service.gov.uk/media/5d7f54c4e5274a27c2c6d53a/smart-meter-roll-out-cost-benefit-analysis-2019.pdf) afirma que se puede estimar entre un 5% y un 35%. Esto coincide con lo informado por GTD que lo establece en un 20%.

In [ ]:
# Cantidad de fallas y duración promedio anual (Datos del proceso VAD 2024-2028, 'Beneficios.xlsx')
# cantidad_fallas_anual = 56855     # Fallas/año
# duracion_promedio_anual = 5.62    # Horas/falla

# Se calcula el ahorro anual por reducción en la duración promedio de las fallas
while True:
    reduccion_duracion_falla = float(input("Reducción en la duración promedio de la falta de suministro [%]: ")) / 100
    if 0 <= reduccion_duracion_falla <= 1:
	    break

costo_de_falla_2025 = 467.19			# USD/MWh

# Energía no suministrada eliminada por año (MWh/año)
energia_no_suministrada_eliminada = (energia_no_suministrada_anual * reduccion_duracion_falla)

beneficio_neto_compensacion_evitada = energia_no_suministrada_eliminada * costo_de_falla_2025 * 2		# USD/año

# Se asigna una parte a cada segmento según su participación en las compensaciones totales
ahorro_por_compensacion_evitada_parcializado = beneficio_neto_compensacion_evitada / ((len(df_operacion) / 5) * 3)  # 3 segmentos regulados
df_operacion["Ahorro por compensación evitada [USD/año]"] = np.where(df_operacion["Segmento"].isin(["Residencial", "No Residencial BT", "No Residencial AT"]),
																	  ahorro_por_compensacion_evitada_parcializado, 0)

# Ahorro global anual por reducción en la duración promedio de las fallas
print(f"Considera la reducción de un {reduccion_duracion_falla * 100}% en la duración promedio de una falla")
print("------------------------------------------------------------------------")
print("Ahorro por compensación evitada (anual): M USD {:,.4f}".format(beneficio_neto_compensacion_evitada / 1_000_000))

### Beneficio 5: Ahorro por efecto de una gestión del consumo

El informe de GTD calcula el beneficion sobre “Gestión de Consumo” suponiendo que, dentro del universo regulado, al 25% de los clientes de mayor consumo (que concentran el 73,92% de la energía vendida) les conviene migrar a una opción horaria y que, gracias al SMI, ese grupo podrá desplazar el 11% de su consumo desde horas punta (Bloque C) a intermedias (Bloque B), valorando ese desplazamiento con un diferencial de 20 USD/MWh entre ambos bloques. Así, el beneficio anual se obtiene multiplicando la energía vendida total del año base del estudio por 0,7392 × 0,11 × 20.

Considerando el año del estudio (2016), se ha ajustado el diferencial de precio por CPI, equivalente a un factor de 1,34.

In [ ]:
# # Ahorro por gestión del consumo energético
# energia_clientes_mayor_consumo = 0.7392
# energia_trasladada = energia_clientes_mayor_consumo * 0.11

# precio_diferencial = 20	* 1.34			# USD/MWh (ajustado por CPI, 2016-2025)

# # Indice para aplicar el cálculo solo a los segmentos Residencial y No_Residencial
# indice_segmento = np.where(df_operacion["Segmento"] == "Residencial", 1,
#                		np.where(df_operacion["Segmento"] == "No_Residencial", 1, 0))

# df_operacion["Ahorro por gestión del consumo [USD/año]"] = (df_operacion["Energía facturada anual [MWh/año]"]
# 															 * indice_segmento
# 															 * energia_trasladada
# 															 * precio_diferencial)

# beneficio_neto_gestion_consumo = df_operacion["Ahorro por gestión del consumo [USD/año]"].sum()

# # Ahorro global anual por gestión del consumo energético
# print("Ahorro por gestión del consumo energético (anual): M USD {:,.4f}"
# 	  .format(beneficio_neto_gestion_consumo / 1_000_000))

La forma de estimar el efecto que tendría implementar bloques horarios de suministro a un precio energético más barato se realiza considerando cuanta energía, y que consumos de ella es posible trasladar de los bloques caros a los bloques baratos.

Para esto se hace uso de la información dispuesta en el siguiente estudio: [Informe: Usos de la Energía en Chile (2018)](https://www.electricas.cl/wp-content/uploads/2020/07/informe-final-usos-energia-chile-2018.pdf). En el se aproxima la responsabilidad del uso energético proveniente de cada artefacto residencial (página 91, Gráfico 10).

En el mismo informe, se dispone de una gráfica (página 33, Gráfico 3) en el que se detalla el porcentaje de clientes que cuentan con un artefacto/equipo en particular.

En sintesis, los equipos o artefactos que en principio podrían desplazar su uso hacia horas de menor consumo son las siguientes:

 - Lavado de Ropa: 1,6% del total de energía, el 98% de los clientes tiene el equipo.
 - Secado de Ropa: 6,2% del total de energía, el 29,9% de los clientes tiene el equipo.
 - Plancha de Ropa: 2,1% del total de energía, el 69,3% de los clientes tiene el equipo.
 - Aspiradora: 5,0% del total de energía, el 53,4% de los clientes tiene el equipo.
 - Hervidor: 4,0% del total de energía, el 77,9% de los clientes tiene el equipo.

También se requiere conocer la curva de demanda característica del cliente residencial, la cual se obtiene a partir del [Modelo CREST](https://figshare.com/articles/dataset/CREST_Demand_Model_v2_0/2001129). De este modelo se estima la proporción de energía consumida en el bloque más caro respecto del total diario.

En cuanto a la reducción del consumo, a modo de referencia se pueden revisar los precios de adjudicación de los diferentes contratos de suministro eléctrico, revisando aquellos puramente provenientes de combustibles fósiles, y aquellos provenientes de energías renovables [Informe: Licitaciones de Suministro](https://icrchile.cl/wp-content/uploads/2023/04/E001-1.pdf). El informe asegura que en promedio, los contratos adjudicados han acumulado una caída de 78% entre 2014 y 2021, lo que se explica en gran medida por las tecnologías de generación que están detrás de dichos contratos.

In [ ]:
# Ahorro por reduccion del precio de la energía gracias a la gestión del consumo
while True:
	reduccion_precio_de_la_energia = float(input("Reducción en el precio de la energía gracias a la gestión del consumo [%]: ")) / 100
	if 0 <= reduccion_precio_de_la_energia <= 1:
	    break

bloque_caro_de_energia = 0.5126		# % de la energía consumida que se concentra en el bloque caro (horario 18:00 a 7:59hrs)
energia_trasladada = 0.20			# % de la energía trasladada del bloque caro al bloque barato

precio_diferencial = precio_energia_usd_mwh * reduccion_precio_de_la_energia		# USD/MWh
	
# Indice para aplicar el cálculo solo a los segmentos Residencial y No_Residencial
indice_segmento = np.where(df_operacion["Segmento"] == "Residencial", 1, 0)

df_operacion["Ahorro por gestión del consumo [USD/año]"] = (df_operacion["Energía facturada anual [MWh/año]"]
																				* indice_segmento
																				* bloque_caro_de_energia
																				* energia_trasladada			# Energía trasladada al bloque barato 
																				* precio_diferencial)

beneficio_neto_gestion_consumo = df_operacion["Ahorro por gestión del consumo [USD/año]"].sum()

# Ahorro global anual por gestión del consumo energético
print(f"Considera la reducción de un {reduccion_precio_de_la_energia * 100}% en el precio de la energía")
print("------------------------------------------------------------------")
print("Ahorro por gestión del consumo energético (anual): M USD {:,.4f}"
	  .format(beneficio_neto_gestion_consumo / 1_000_000))

### Beneficio 6: Ahorro por efecto de reducción de la cuenta a pagar de energía dada la reducción del consumo de los clientes finales

En cuanto a este tipo de beneficio, se puede contabilizar a través del ahorro generado en la cuenta final de los clientes producto de su reducción en el consumo eléctrico. Este fenómeno se ha estudiado bastante, ejemplos concretos se decriben en [Despliegue UK](https://assets.publishing.service.gov.uk/media/5d7f54c4e5274a27c2c6d53a/smart-meter-roll-out-cost-benefit-analysis-2019.pdf), en el cual afirman que:

 - La *European Smart Metering Industry Group* (ESMIG), realizó pilotos de pruebas en los cuales se encontró una reducción promedio del consumo en torno a un 5,4%. Mientras que con feedback del consumo en tiempo real se redujo en promedio un 7,9%.
 - Otros estudios como el de *Energy Demand Research Project* (EDRP) estimaron una reducción en torno al 3% en promedio.
 - Finalmente, en el estudio de despligue de UK se quedan con una reducción promedio del 3% (para es caso residencial) y del 2,8% (para es caso no residencial), aclarando que estos porcentajes podrían subir con el tiempo a medida que la tecnología va madurando.

Para estimar el ahorro que supondría, se tomará como precio de la energía el PNP equivalente nacional calculado en el proceso PNP Julio 2025 ([Anexos PNP Julio 2025](https://comisionenergia-my.sharepoint.com/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fonedrive%2Dsubdeptargedis%5Fcne%5Fcl%2FDocuments%2FPublicaciones%20WEB%2FPNP%2F2025%2D02%2F02%20ITD%2F4%2E%2DModelo&viewid=f5258698%2D4137%2D4785%2Da047%2D925c3d159fd9&ga=1)).

In [ ]:
# Estimación del ahorro anual por reducción del consumo
while True:
	reduccion_consumo_residencial = float(input("Reducción del consumo de clientes Residenciales [%]: ")) / 100
	reduccion_consumo_no_residencial = float(input("Reducción del consumo de clientes No Residenciales [%]: ")) / 100
	if (0 <= reduccion_consumo_residencial <= 1 and 0 <= reduccion_consumo_no_residencial <= 1):
		break

# reducción anual según segmento: Residendial y No_Residencial
reduccion_consumo = np.where(df_operacion["Segmento"] == "Residencial", reduccion_consumo_residencial,
               			np.where(df_operacion["Segmento"] == "No Residencial BT", reduccion_consumo_no_residencial,
						  	np.where(df_operacion["Segmento"] == "No Residencial AT", reduccion_consumo_no_residencial, 0)))

# Se calcula Energía facturada anual [MWh/año] * reducción consumo [%] * precio energía [USD/MWh]
df_operacion["Ahorro reducción del consumo [USD/año]"] = (df_operacion["Energía facturada anual [MWh/año]"]
                                                          * reduccion_consumo 
                                                          * precio_energia_usd_mwh)

beneficio_neto_reduccion_consumo = df_operacion["Ahorro reducción del consumo [USD/año]"].sum()

print("Considera la reducción de un:", 
	  f"\n -> {reduccion_consumo_residencial * 100}% en el consumo de energía Residencial",
	  f"\n -> {reduccion_consumo_no_residencial * 100}% en el consumo de energía No Residencial")
print("-----------------------------------------------------")
print("Ahorro por reducción del consumo (anual): M USD {:,.4f}"
      .format(beneficio_neto_reduccion_consumo / 1_000_000))

### Beneficio 7: Ahorro por efecto de reducción de emisiones dada la reducción del consumo de los clientes finales

Asociado con la reducción del consumo de los clientes finales, esto tiene consecuencia directa en la reducción de emisiones producto de la reducción de unidades de energía eléctrica generadas.

Este beneficio entonces puede ser calculado a través de datos de Ministerio del Medio Ambiente para conocer la euivalencia entre energía generada y emisiones producidas, [Estudio: Emisiones por kWh](https://huellachile.mma.gob.cl/wp-content/uploads/2024/11/HuellaChile-DCC-Factores-de-emision-nivel-basico_v3.pdf).

Además se puede hacer uso del costo social de emitir unidades de CO2 publicado por el Ministerio de Desarrollo Social, [Costo de emisiones de CO2](https://www.desarrollosocialyfamilia.gob.cl/noticias/nueva-actualizacion-del-precio-social-del-carbono-busca-incentivar-proyectos-publicos-que-reduzcan-l).

In [ ]:
# Ahorro nacional anual por reducción de emisiones de CO2
# reducción consumo anual según segmento: Residendial y No_Residencial
reduccion_consumo = np.where(df_operacion["Segmento"] == "Residencial", reduccion_consumo_residencial,
               			np.where(df_operacion["Segmento"] == "No Residencial BT", reduccion_consumo_no_residencial,
						  	np.where(df_operacion["Segmento"] == "No Residencial AT", reduccion_consumo_no_residencial, 0)))

df_operacion["Ahorro reducción emisiones CO2 [USD/año]"] = (df_operacion["Energía facturada anual [MWh/año]"]
															* reduccion_consumo
                                                            * emisiones_co2_por_mwh
															* costo_social_co2)

beneficio_neto_reduccion_emisiones_co2 = df_operacion["Ahorro reducción emisiones CO2 [USD/año]"].sum()

print("Considera la reducción de un:", 
	  f"\n -> {reduccion_consumo_residencial * 100}% en el consumo de energía Residencial",
	  f"\n -> {reduccion_consumo_no_residencial * 100}% en el consumo de energía No Residencial")
print("------------------------------------------------------------")
print("Ahorro por reducción de emisiones de CO2(anual): M USD {:,.4f}"
      .format(beneficio_neto_reduccion_emisiones_co2 / 1_000_000))

### Beneficio 8: Ahorro por efecto de la reducción del uso de la atención al cliente de las empresas distribuidoras

Este beneficio se obtiene gracias a las reducciones del servicio de atención al cliente debido a la reducción de las estimaciones de consumo, precisión del medidor inteligente, detección automática de fallas e incidencias, mayor información relativa a su consumo personal, etc. Este beneficio se estima, de parte de [Despliegue UK](https://assets.publishing.service.gov.uk/media/5d7f54c4e5274a27c2c6d53a/smart-meter-roll-out-cost-benefit-analysis-2019.pdf), en una reducción de las llamadas y comunicaciones con el servicio en torno a un 60%.

Para construir este beneficio para el caso nacional, se hace uso de los resultados dispuestos en [Anexo 4-1, VAD 2024-2028](https://comisionenergia-my.sharepoint.com/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fonedrive%2Dsubdeptargedis%5Fcne%5Fcl%2FDocuments%2FPublicaciones%20WEB%2FVAD%2FVAD%202024%2D2028%2F02%20Informe%20T%C3%A9cnico%20Preliminar%2FATD1%2FAnexos%2FAnexo%204%2FAnexo%204%2D1&viewid=f5258698%2D4137%2D4785%2Da047%2D925c3d159fd9&ga=1). Del cual se puede obtener un costo promedio nacional de la atención por cliente al año.

In [ ]:
# Ahorros asociados al corte y reposición remotos de suministro
while True:
	reduccion_atencion_clientes = float(input("Reducción en el uso de la atención al cliente [%]: ")) / 100
	if 0 <= reduccion_atencion_clientes <= 1:
	    break

# Costo promedio de atención al cliente (Fuente: VAD 2024-2028)
atencion_al_cliente = 2.4662		# USD/cliente-año

# Indice para aplicar el cálculo solo al segmento Regulado (Residencial y No Residencial)
indice_segmento = np.where(df_operacion["Segmento"] == "Residencial", 1,
			   		np.where(df_operacion["Segmento"] == "No Residencial BT", 1,
				 		np.where(df_operacion["Segmento"] == "No Residencial AT", 1, 0)))

df_operacion["Ahorro atención al cliente [USD/año]"] = (df_operacion["Cantidad de clientes"]
														* atencion_al_cliente
														* reduccion_atencion_clientes
														* indice_segmento)

beneficio_neto_atencion_cliente = df_operacion["Ahorro atención al cliente [USD/año]"].sum()

# Ahorro global anual por reducción en la atención al cliente
print(f"Considera la reducción de un {reduccion_atencion_clientes * 100}% en el uso de atención al cliente")
print("-------------------------------------------------------------------")
print("Ahorro por reducción en atención al cliente (anual): M USD {:,.4f}"
	  .format(beneficio_neto_atencion_cliente / 1_000_000))

### Beneficio 9: Ahorro por efecto de la reducción en los requerimientos de inversión de red

Este beneficio tiene que ver con los ahorros generados por la reducción de requerimientos de inversión. Esto se dá gracias a la mejora en la visualización de las redes y el monitoreo de estas, con esta tecnología se puede saber con mucha más precisión las inversiones de red que realmente son necesarias.

Para calcular el beneficio se hace uso de las tablas de inversión de red dispuestas en el [Anexo 5, VAD 2024-2028](https://comisionenergia-my.sharepoint.com/:x:/r/personal/onedrive-subdeptargedis_cne_cl/_layouts/15/Doc.aspx?sourcedoc=%7BCFFC61F6-6AC0-40D6-B74E-3191D176ED1C%7D&file=Anexo%205-VAD%20ATD12.xlsx&action=default&mobileredirect=true). Con ellas se obtiene un monto neto nacional de inversión asociada a cumplir los requerimientos de calidad de la NTD.

En cuanto al ahorro, [Despliegue UK](https://assets.publishing.service.gov.uk/media/5d7f54c4e5274a27c2c6d53a/smart-meter-roll-out-cost-benefit-analysis-2019.pdf) indica que se pueden reducir la inversión entre un 5% y un 10%, incluso más en determinadas circunstancias

In [ ]:
# Ahorros sobre la inversión evitada en infraestructura de red
while True:
	reduccion_infraestructura = float(input("Reducción en la inversión de infraestructura de red [%]: ")) / 100
	if 0 <= reduccion_infraestructura <= 1:
	    break

# Costo promedio de infraestructura de red (Fuente: VAD 2024-2028)
inversion_infraestructura = 199701412		# USD/año

# Inversión evitada
inversion_evitada = inversion_infraestructura * reduccion_infraestructura

# Se asigna una parte a cada segmento según su participación en la inversión total
ahorro_por_inversion_evitada_parcializada = inversion_evitada / (len(df_operacion) / 5)
df_operacion["Ahorro por inversión evitada [USD/año]"] = np.where(df_operacion["Segmento"].isin(["Red"]),
																	  ahorro_por_inversion_evitada_parcializada, 0)

# Ahorro global anual por reducción en la inversión de infraestructura de red
print(f"Considera la reducción de un {reduccion_infraestructura * 100}% en la inversión de infraestructura de red")
print("---------------------------------------------------------------------------")
print("Ahorro por inversión evitada en infraestructura de red (anual): M USD {:,.4f}"
	  .format(inversion_evitada / 1_000_000))

### Beneficio Neto

In [ ]:
df_operacion["Beneficio total anual [USD/año]"] = (df_operacion["Ahorro en lectura pedestre [USD/año]"]				# Beneficio 1
                                                   + df_operacion["Ahorro corte y reposición remoto [USD/año]"]		# Beneficio 2
                                                   + df_operacion["Ahorro reducción de PNT [USD/año]"]				# Beneficio 3
                                                   + df_operacion["Ahorro por compensación evitada [USD/año]"]		# Beneficio 4
                                                   + df_operacion["Ahorro por gestión del consumo [USD/año]"]		# Beneficio 5
												   + df_operacion["Ahorro reducción del consumo [USD/año]"]			# Beneficio 6
                                                   + df_operacion["Ahorro reducción emisiones CO2 [USD/año]"]		# Beneficio 7
												   + df_operacion["Ahorro atención al cliente [USD/año]"]			# Beneficio 8
												   + df_operacion["Ahorro por inversión evitada [USD/año]"])		# Beneficio 9

print("Beneficio total anual del proyecto (dada la configuración de tecnologías y supuestos): M USD {:,.4f}"
	  .format(df_operacion["Beneficio total anual [USD/año]"].sum() / 1_000_000))